In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('fede'))
if module_path not in sys.path:
    sys.path.append(module_path)
# import pandas as pd
from client import Client
from supported_modles import Supported_modles
import utils
from fedavg import Fedavg
from sklearn.utils.class_weight import compute_sample_weight
import numpy as np
import random

In [2]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [3]:
selected_model = Supported_modles.NN_classifier

In [4]:
clients = utils.set_data_mock(selected_model, True, False)

Creating node1.
Creating node2.
Creating node3.
DOWNSAMLING node3


In [5]:
f1_before = []
for client in clients:
    client.init_empty_model(0.01)
    client.train_model(epochs=50)
    try: 
        score = client.test_model_f1()
    except IndexError:
        score = 0
    f1_before.append(score)
    print(score)

0.9754584326454347


KeyboardInterrupt: 

In [6]:
fedavg = Fedavg("global", selected_model)

Waitiing for a Connection..


In [7]:
###First learn model on clients:
round_weights = []
dataset_size = 0
applicable_models = []
first = True
epochs = 10

for client in clients:
    client.init_empty_model(0.01)
    X_train, X_test, y_train, y_test = client.split_data(0.8)

    client.train_model(X_train,y_train,epochs=epochs)
    dataset_size += client.x.shape[0]
    round_weights.append(dataset_size)
    applicable_models.append(client.model)
    if first:
        fedavg.init_global_model(client.model)
        first = False
    print('.')

round_weights = np.array(round_weights) / dataset_size # calculate weight based on actual dataset size
fedavg.update_global_model(applicable_models, round_weights, selected_model)

.
.
.


In [8]:
number_of_rounds = 9
epochs = 10
max_score = 0
optimal_model = None


for round in range(number_of_rounds):
    print(round +1, end=' ')

    applicable_clients = random.sample((clients), len(clients))#random.randint(1, 4))
    applicable_models = []
    applicable_name = []
    round_weights = []
    dataset_size = 0
    

    for client in applicable_clients:
        print(f'.', end='')

        X_train, X_test, y_train, y_test = client.split_data(0.8)
       
        dataset_size += X_train.shape[0]
        sample_weights = compute_sample_weight('balanced', y=y_train)

        client.load_global_model(fedavg.model)
        client.train_local_agent(X_train, y_train, epochs, sample_weights)
        round_weights.append(X_train.shape[0])
        applicable_models.append(client.model)
        print(round_weights)


    round_weights = np.array(round_weights) / dataset_size
    fedavg.update_global_model(applicable_models, round_weights, selected_model)

    for client in clients:
        score = fedavg.test_model_f1(y_test=client.y_test, X_test=client.x_test)
        print(score)

1 .[29414]
.[29414, 31106]
.[29414, 31106, 2570]
0.8123869675301189
0.6977309408525586
0.9579288025889968
2 .[31106]
.[31106, 2570]
.[31106, 2570, 29414]
0.9367877082083698
0.8371523333746198
0.9223040857334226
3 .[31106]
.[31106, 29414]
.[31106, 29414, 2570]
0.9343957032289121
0.9243940476964065
0.9873280231716147
4 .[29414]
.[29414, 2570]
.[29414, 2570, 31106]
0.9410026543797266
0.9395171957671957
0.9645125140082181
5 .[29414]
.[29414, 31106]
.[29414, 31106, 2570]
0.9527524924143911
0.9426215938814025
0.9835043988269795
6 .[31106]
.[31106, 29414]
.[31106, 29414, 2570]
0.9760554732926433
0.9422765518336244
0.9874236435501258
7 .[29414]
.[29414, 2570]
.[29414, 2570, 31106]
0.9709646446845489
0.9661750561077268
0.9942112879884225
8 .[2570]
.[2570, 29414]
.[2570, 29414, 31106]
0.982995484676722
0.9705598765828888
0.9824881430134986
9 .[2570]
.[2570, 29414]
.[2570, 29414, 31106]
0.9771101227883592
0.9715103839904766
0.9845201238390092
